In [6]:
import pandas as pd
import sys
import os
from pathlib import Path

project_root = str(Path.cwd().parent)
sys.path.append(project_root)
from sklearn.model_selection import train_test_split
from src.data.make_dataset import extract_departure_time
from src.data.make_dataset import impute_distance_by_average
from src.data.make_dataset import clean_cabin_type
from src.data.make_dataset import clean_data
from src.models.train_model import train_model
from src.models.predict_model import predict_model

In [8]:
airport_combined_path = '../data/processed/all_airports.csv'

In [9]:
airport_df = pd.read_csv(airport_combined_path)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
airport_df.sample(10)

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
6143661,28a9210240792381cd92c5b70b8b1041,2022-05-02,2022-05-16,IAD,DFW,PT3H18M,False,False,True,327.60,1177.0,1652737020,2022-05-16T17:37:00.000-04:00,1652748900,2022-05-16T19:55:00.000-05:00,DFW,IAD,United,UA,Airbus A319,11880,1177,coach
4271857,c3261ed60fd0827739cbc2bf7697fd8f,2022-05-12,2022-05-31,DFW,JFK,PT3H41M,False,False,True,328.60,1380.0,1654026360,2022-05-31T14:46:00.000-05:00,1654039620,2022-05-31T19:27:00.000-04:00,JFK,DFW,American Airlines,AA,Boeing 737-800,13260,1380,coach
2581352,40d66d0b8e92e279a868e6fc110ea835,2022-05-10,2022-05-22,CLT,DTW,PT7H24M,False,False,False,366.10,1030.0,1653219060||1653238500,2022-05-22T07:31:00.000-04:00||2022-05-22T12:5...,1653226140||1653245700,2022-05-22T09:29:00.000-04:00||2022-05-22T14:5...,LGA||DTW,CLT||LGA,Delta||Delta,DL||DL,Embraer 175||Airbus A319,7080||7200,545||485,coach||coach
9975066,dc97d6c665a9347f7f98b6b07a6e439f,2022-05-02,2022-06-09,MIA,LGA,PT8H30M,False,False,False,232.60,2495.0,1654774200||1654792260,2022-06-09T07:30:00.000-04:00||2022-06-09T11:3...,1654785240||1654804800,2022-06-09T09:34:00.000-05:00||2022-06-09T16:0...,DFW||LGA,MIA||DFW,American Airlines||American Airlines,AA||AA,Airbus A321||Boeing 737-800,11040||12540,1115||1380,coach||coach
8490621,a4bf5037b7942184380a0bb55d6035bf,2022-04-17,2022-05-11,LGA,IAD,PT8H23M,True,False,False,167.20,1303.0,1652299800||1652323860,2022-05-11T16:10:00.000-04:00||2022-05-11T22:5...,1652309220||1652329980,2022-05-11T18:47:00.000-04:00||2022-05-12T00:3...,ATL||IAD,LGA||ATL,Delta||Delta,DL||DL,Airbus A321||Airbus A320,9420||6120,762||541,coach||coach
7654035,1b003c8fb6abbb2d17934178b7cd9683,2022-04-28,2022-06-13,LAX,JFK,PT5H47M,False,False,True,459.35,2458.0,1655133000,2022-06-13T08:10:00.000-07:00,1655153820,2022-06-13T16:57:00.000-04:00,JFK,LAX,American Airlines,AA,Airbus A321,20820,2458,coach
2666103,c10ee5f67930f8b56311679dddef5d26,2022-05-14,2022-05-23,CLT,SFO,PT7H32M,False,False,False,667.10,2439.0,1653341280||1653352500,2022-05-23T17:28:00.000-04:00||2022-05-23T19:3...,1653350100||1653368400,2022-05-23T18:55:00.000-05:00||2022-05-23T22:0...,ORD||SFO,CLT||ORD,United||United,UA||UA,Embraer 175 (Enhanced Winglets)||Boeing 777,8820||15900,592||1847,coach||coach
13293595,9fba6525a7297ada0a15447fe488ca58,2022-05-08,2022-05-19,SFO,CLT,PT9H49M,False,False,False,700.59,3111.0,1653029700||1653057900,2022-05-19T23:55:00.000-07:00||2022-05-20T10:4...,1653049140||1653065040,2022-05-20T08:19:00.000-04:00||2022-05-20T12:4...,EWR||CLT,SFO||EWR,United||United,UA||UA,BOEING 777-300ER||,19440||7140,2566||545,coach||coach
9506601,248e508744811fea28505688ba14146b,2022-04-17,2022-05-09,MIA,OAK,PT10H30M,False,False,False,391.58,NaN,1652103600||1652135940,2022-05-09T09:40:00.000-04:00||2022-05-09T15:3...,1652122680||1652141400,2022-05-09T11:58:00.000-07:00||2022-05-09T17:1...,LAS||OAK,MIA||LAS,Spirit Airlines||Spirit Airlines,NK||NK,||,19080||5460,None||None,coach||coach
3265763,7211f77fec0af9555b3e02428aa6c95e,2022-05-06,2022-06-23,DEN,SFO,PT11H38M,False,False,False,384.70,1172.0,1655983920||1655993100||1656020100,2022-06-23T05:32:00.000-06:00||2022-06-23T08:0...,1655989500||1655998140||1656025800,2022-06-23T07:05:00.000-06:00||2022-06-23T08:2...,SLC||LAS||SFO,DEN||SLC||LAS,Delta||Delta||Alaska Airlines,DL||DL||AS,Airbus A319||Boeing 737-900||Airbus A320,5580||5040||5700,380||368||424,coach||coach||coach


In [6]:
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13519999 entries, 0 to 13519998
Data columns (total 23 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   legId                              object 
 1   searchDate                         object 
 2   flightDate                         object 
 3   startingAirport                    object 
 4   destinationAirport                 object 
 5   travelDuration                     object 
 6   isBasicEconomy                     bool   
 7   isRefundable                       bool   
 8   isNonStop                          bool   
 9   totalFare                          float64
 10  totalTravelDistance                float64
 11  segmentsDepartureTimeEpochSeconds  object 
 12  segmentsDepartureTimeRaw           object 
 13  segmentsArrivalTimeEpochSeconds    object 
 14  segmentsArrivalTimeRaw             object 
 15  segmentsArrivalAirportCode         object 
 16  segmentsDepartur

In [7]:
import pandas as pd

def calculate_average_distances(airport_df):
    """
    Calculates the average distance between unique origin and destination airports.

    Args:
        airport_df: A pandas DataFrame with 'Origin', 'Destination', and 'Distance' columns.

    Returns:
        A pandas DataFrame with unique origin and destination airports and their average distance.
    """
    average_distances = airport_df.groupby(['startingAirport', 'destinationAirport'])['totalTravelDistance'].mean().reset_index()

    return average_distances

In [8]:
average_distances_df = calculate_average_distances(airport_df)
print(average_distances_df)

    startingAirport destinationAirport  totalTravelDistance
0               ATL                BOS           998.032995
1               ATL                CLT           662.624730
2               ATL                DEN          1546.128489
3               ATL                DFW           907.045841
4               ATL                DTW           973.100301
5               ATL                EWR           828.117167
6               ATL                IAD           621.642341
7               ATL                JFK           962.933895
8               ATL                LAX          2086.698655
9               ATL                LGA           869.946287
10              ATL                MIA          1146.822334
11              ATL                OAK          2422.447664
12              ATL                ORD           963.911324
13              ATL                PHL           872.197084
14              ATL                SFO          2480.410744
15              BOS                ATL  

In [10]:
Airport_df_cleaned = clean_data(airport_df)

In [10]:
Airport_df_cleaned.head()

,startingAirport,destinationAirport,Departure_Year,Departure_Month,Departure_Day,depatureTimeCategory,Cabin_Type,Cabin_Score,totalFare,totalTravelDistance
0,ATL,BOS,2022,4,17,Afternoon,coach,0,248.6,947.0
1,ATL,BOS,2022,4,17,Morning,coach,0,248.6,947.0
2,ATL,BOS,2022,4,17,Afternoon,coach,0,248.6,947.0
3,ATL,BOS,2022,4,17,Evening,coach,0,248.6,947.0
4,ATL,BOS,2022,4,17,Afternoon,coach,0,248.6,947.0


In [11]:
Airport_df_cleaned.columns

Index(['startingAirport', 'destinationAirport', 'Departure_Year',
       'Departure_Month', 'Departure_Day', 'depatureTimeCategory',
       'Cabin_Type', 'Cabin_Score', 'totalFare', 'totalTravelDistance'],
      dtype='object')

In [11]:
X = Airport_df_cleaned.drop(['totalFare'], axis=1)  # remove target variable
y = Airport_df_cleaned['totalFare']

In [35]:
airport_df.head()


,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,False,True,248.6,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,248.6,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,248.6,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,False,True,248.6,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,False,True,248.6,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach


In [40]:
X['Cabin_Score'].unique()

array([0, 4, 3], dtype=int64)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2,  # 20% for testing
    random_state=42  # for reproducibility
)

In [13]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [19]:
model = lgb.LGBMRegressor(
    objective='regression',
    random_state=42,
    n_jobs=-1
)

In [20]:
pipeline = train_model(model, X_train, y_train, output_path='../models/')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.114047 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 45
[LightGBM] [Info] Number of data points in the train set: 10815999, number of used features: 8
[LightGBM] [Info] Start training from score 373.752004


In [21]:
results = predict_model(X_test, y_test, pipeline)


Model Performance:
RMSE: $201.06
MAE: $148.90
R2 Score: 0.062
MAPE: 57.97%


In [25]:
import xgboost as xgb

In [29]:
 from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    )

In [30]:
pipeline = train_model(model, X_train, y_train, output_path='../models/')

In [32]:
results = predict_model(X_test, y_test, pipeline)


Model Performance:
RMSE: $201.00
MAE: $148.83
R2 Score: 0.062
MAPE: 57.87%


In [22]:
from src.models.train_model import tune_pipeline

In [23]:
param_grid = {
    'model__learning_rate': [0.1,0.001],
    'model__n_estimators': [1000,1500,2000,2500],
    'model__num_leaves': [31, 62, 127,200],
    'model__max_depth': [-1, 5, 10],
    'model__min_child_samples': [20, 50, 100],
    'model__subsample': [0.6, 0.8, 1.0],
    'model__colsample_bytree': [0.6, 0.8, 1.0]
}

# Tune the pipeline
results = tune_pipeline(
    pipeline=pipeline,
    param_grid=param_grid,
    X_train=X_train,
    y_train=y_train,
    n_iter= 5,
    search_type='random',  # or 'random'
    cv=5
)

# Get the best pipeline
best_pipeline = results['best_pipeline']


Starting random search CV...
Fitting 5 folds for each of 5 candidates, totalling 25 fits


KeyboardInterrupt: 

In [1]:
import joblib

In [4]:
import sys
from pathlib import Path

# Add your project root to Python path
project_root = str(Path.cwd().parent)  # Adjust this based on your current location
if project_root not in sys.path:
    sys.path.append(project_root)

# Now load the pipeline
best_pipeline = joblib.load('../models/best_pipeline_random_20241101_184558.joblib')

In [14]:
results_tune = predict_model(X_test, y_test, best_pipeline)


Model Performance:
RMSE: $201.00
MAE: $148.83
R2 Score: 0.062
MAPE: 57.87%
